<a href="https://colab.research.google.com/github/hosseinfani/Vector-Semantics-Evaluation/blob/main/Topk_Similar_Words_Bug_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

from collections import defaultdict
def readfile(filepath, indexes=None):
    if not indexes:
        indexes = [0, 1, 2]
    file1 = open(filepath)
    count = 0
    dictionary = defaultdict(dict)

    while True:
        count += 1
        line = file1.readline()
        if not line:
            break
        file_tokens = line.strip().split()
        dictionary[str(file_tokens[indexes[0]])][str(file_tokens[indexes[1]])] = file_tokens[indexes[2]]

    file1.close()
    return dictionary


In [13]:
def getmapandndcg(ground_truth_model, w2v):
    # keys =
    # map = 0
    # ndcg = 0
    # total = 0
    # exists = 0
    for key in list(ground_truth_model.keys()):
        print(f"For word {key}:")
        # sims = dict()
        try:
            # for sim in w2v.most_similar(key, topn=10):
            #     sims[sim[0]] = round(sim[1] * 10, 2)

            # sims = normalize_dict(sims,1,10)
            ground_truth = ground_truth_model[key]
            change = True

            if ground_truth.__len__() < 10 and ground_truth.__len__() != 0:
                while change:
                    length = len(list(ground_truth.keys()))
                    change = False
                    for key in list(ground_truth.keys()):
                        newvalues = ground_truth_model[key]
                        if list(newvalues.keys()) not in list(ground_truth.keys()):
                            change = True
                            ground_truth.update(ground_truth_model[key])
                    if len(ground_truth.keys()) == length:
                        break
            sorted_result = dict(sorted(ground_truth.items(), key=lambda x: x[1]))
            ground_truth = {k: sorted_result[k] for k in list(sorted_result)[:10]}
            for key, value in ground_truth.items():
                ground_truth[key] = round(float(value))
            
            print(ground_truth)

            # qrel = {
            #     'q1': ground_truth,
            # }

            # run = {
            #     'q1': sims,
            # }
            # evaluator = pytrec_eval.RelevanceEvaluator(
            #     qrel, {'map', 'ndcg'})
            # print(f"ground_truth is {ground_truth}")
            # print(f"prediction is {sims}")
            # print(json.dumps(evaluator.evaluate(run), indent=1))
            # map = map + evaluator.evaluate(run)['q1']['map']
            # ndcg = ndcg + evaluator.evaluate(run)['q1']['ndcg']
            # exists = exists + 1
            # if evaluator.evaluate(run)['q1']['map'] > 0:
            #     total = total + 1
            #     # if evaluator.evaluate(run).q1.map !=0 or evaluator.evaluate(run).q1.ndcg !=0 :
            # #     result = evaluator.evaluate(run)
        except Exception as e:
            pass
            print(repr(e))

    # print(f"Map is {map}")
    # print(f"NDCG is {ndcg}")
    #return map, ndcg, total, exists

In [22]:
# sample.txt
# a	b	7.35
# a	c	10.00
# b	d	6
# c	e	6.31
# e	f	6.77
filepath = "/content/sample.txt"
wordsim_353 = readfile(filepath)
getmapandndcg(wordsim_353, None)

# For word a:
# {'c': 10, 'd': 6, 'e': 6, 'f': 7, 'b': 7}
# For word b:
# {'d': 6} => should have 'a','c','e', .... because a -> b also b -> a and all similar to a gonna be in b too.
# For word c:
# {'e': 6, 'f': 7} 
# For word e:
# {'f': 7}

For word a:
{'c': 10, 'e': 6, 'f': 7, 'd': 7, 'b': 7}
For word b:
{'d': 7}
For word c:
{'e': 6, 'f': 7}
For word e:
{'f': 7}


In [20]:
import networkx as nx
def readfile_hossein(filepath, indexes=None, header=False):
    g = nx.Graph()
    if not indexes:
        indexes = [0, 1, 2]
    with open(filepath) as file1:
        if header: file1.readline()
        while True:
            line = file1.readline()
            if not line: break
            file_tokens = line.strip().split()
            word1, word2, score = str(file_tokens[indexes[0]]), str(file_tokens[indexes[1]]), float(file_tokens[indexes[2]])
            if score > 6:
                g.add_edges_from([(word1, word2, {'weight': score})])

    top_sim = dict()
    for word in g.nodes():
        top_sim[word] = {x:1 for x in nx.node_connected_component(g, word) if x != word}

    return top_sim

In [21]:
filepath = "/content/sample.txt"
wordsim_353 = readfile_hossein(filepath)
print(wordsim_353)

{'a': {'d': 1, 'e': 1, 'b': 1, 'c': 1, 'f': 1}, 'b': {'d': 1, 'e': 1, 'c': 1, 'f': 1, 'a': 1}, 'c': {'d': 1, 'e': 1, 'b': 1, 'f': 1, 'a': 1}, 'd': {'e': 1, 'b': 1, 'c': 1, 'f': 1, 'a': 1}, 'e': {'d': 1, 'b': 1, 'c': 1, 'f': 1, 'a': 1}, 'f': {'d': 1, 'e': 1, 'b': 1, 'c': 1, 'a': 1}}
